In [3]:
import cv2
import numpy as np
from Util import get_parking_spots_bboxes, empty_or_not
import warnings 
warnings.filterwarnings('ignore')

In [5]:
# Load Video
cap = cv2.VideoCapture('parking_1920_1080.mp4')
if not cap.isOpened():
    print('Error: Could not open the Video.')
    exit()
    
#Load mask
mask=cv2.imread('mask_1920_1080.png', 0) # read binary
if mask is None:
    print("Error: Mask could not load mask")
    exit()


#Calc The deffrence between the frist frame & seconed frame
def calc_diff(frame1, frame2):
    return np.abs(np.mean(frame1) - np.mean(frame2))
previous_frame = None # for store previous frame 

#Connect components of the mask in the img
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S) # for analysis the mask and take the coordinates.
boxes = get_parking_spots_bboxes(connected_components) # func to select the bboxes 
boxes_status = [None for j in boxes] # list to store the box status
diff = [None for j in boxes] # List to store The deffrence between the frist frame & seconed frame
step= 40
img_num = 0

while True:
    success, img = cap.read()
    if not success:
        print('End of Video')
        break

    # This to calc the diffrence between frames
    if img_num % step == 0 and previous_frame is not None: # update frame and to make sure that already have previous frame to calc the diffrence.
        for box_index, box in enumerate(boxes):
            x1, y1, w, h= box
            box_crop = img[y1:y1 + h, x1:x1 + w, :] # to crop the img(x-axis, y-axis) and select RGB
            diff[box_index] = calc_diff(box_crop, previous_frame[y1:y1 + h, x1:x1 + w, :]) # this crop the box in currently frame and compare with the previous frame

    if img_num % step == 0:
        if previous_frame is None:
            arr = range(len(boxes)) # make indexes to start the comparisons.
        else:
            arr = [j for j in np.argsort(diff) if diff[j] / np.amax(diff) > 0.4] # that make Ascending order and the goal of this to update the boxes that have changed up to 40%
                                                                                                      
        for box_index in arr:
            box = boxes[box_index] # take the coordinates of the current box
            x1, y1, w, h= box
            box_crop = img[y1:y1 + h, x1:x1 + w, :]
            box_status = empty_or_not(box_crop)
            boxes_status[box_index] = box_status

    if img_num % step == 0:
        previous_frame = img.copy() # to store the currently frame as the previous frame after finish the process.


    for box_index, box in enumerate(boxes):
        box_status = boxes_status[box_index]
        x1, y1, w, h = boxes[box_index]   
        if box_status:
            cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (0, 255, 0), 2)
        else:
            cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (0, 0, 255), 2)

    cv2.rectangle(img, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(img, 'Available Spots: {} / {}'.format(str(sum(boxes_status)), str(len(boxes_status))), 
                (100, 60),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('image', img)

    # Wait for 1 millisecond and check if the user pressed the 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    img_num+= 1
# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

End of Video
